### Author: Zorah Zafari 
### Date: 01/26/2025
### This code opens each URL and scrapes data from each individual firm 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv

# Set up the Selenium WebDriver
driver = webdriver.Chrome()

def get_firm_urls(page_url):
    """
    Fetches all firm URLs from a given page using Selenium to handle CAPTCHA and dynamic content.
    """
    driver.get(page_url)
    print(f"Accessing page: {page_url}")
    
    # Wait for user to solve CAPTCHA manually, if prompted
    input("If prompted, please solve the CAPTCHA and press Enter to continue...")
    time.sleep(5)  # Allow additional time for the page to fully load

    # Parse the rendered page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    firm_urls = []
    for link in soup.find_all('a', href=True):
        if '/profile/' in link['href']:  # Adjust pattern as necessary for the specific structure
            full_url = 'https://www.bbb.org' + link['href']
            print(f"Found URL: {full_url}")  # Debugging output to verify URLs
            firm_urls.append(full_url)
    return firm_urls

def scrape_firm_details(firm_url):
    """
    Extracts specific details from a firm's page.
    """
    driver.get(firm_url)
    print(f"Scraping data from: {firm_url}")
    time.sleep(3)  # Allow time for the page to fully load

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        # Extract data based on the provided structure
        name = soup.find('span', class_='bds-h2 bpr-header-business-name', id='businessName').text.strip() if soup.find('span', class_='bds-h2 bpr-header-business-name', id='businessName') else 'N/A'
        address = ' '.join([p.text.strip() for p in soup.find('div', class_='bpr-overview-address').find_all('p', class_='bds-body')]) if soup.find('div', class_='bpr-overview-address') else 'N/A'
        website = soup.find('a', href=True, rel='nofollow noreferrer')['href'] if soup.find('a', href=True, rel='nofollow noreferrer') else 'N/A'
        business_started = soup.find('dt', text='Business Started Locally:').find_next_sibling('dd').text.strip() if soup.find('dt', text='Business Started Locally:') else 'N/A'
        phone = soup.find('a', href=True, text=lambda x: x and x.startswith('(')).text.strip() if soup.find('a', href=True, text=lambda x: x and x.startswith('(')) else 'N/A'
    except Exception as e:
        print(f"Error scraping details from {firm_url}: {e}")
        return {'URL': firm_url, 'Name': 'N/A', 'Address': 'N/A', 'Website': 'N/A', 'Business Started': 'N/A', 'Phone': 'N/A'}

    return {'URL': firm_url, 'Name': name, 'Address': address, 'Website': website, 'Business Started': business_started, 'Phone': phone}

# Main Scraping Loop
base_url = 'https://www.bbb.org/us/category/payday-loans?page='
all_firm_details = []

for page in range(1, 16):  # Loop through pages 1 to 15
    page_url = base_url + str(page)
    firm_urls = get_firm_urls(page_url)

    for firm_url in firm_urls:
        details = scrape_firm_details(firm_url)
        all_firm_details.append(details)
        time.sleep(2)  # Delay between requests to each firm's page

# Save extracted data to a CSV file
csv_filename = 'firm_details.csv'
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['URL', 'Name', 'Address', 'Website', 'Business Started', 'Phone'])
    writer.writeheader()  # Write header row
    writer.writerows(all_firm_details)  # Write rows of firm details

print(f"Total firms scraped: {len(all_firm_details)}")
print(f"Details saved to {csv_filename}")

# Close the WebDriver when done
driver.quit()


Accessing page: https://www.bbb.org/us/category/payday-loans?page=1


If prompted, please solve the CAPTCHA and press Enter to continue... 


Found URL: https://www.bbb.org/us/wi/wausau/profile/payday-loans/xpress-cash-0694-44153054
Found URL: https://www.bbb.org/us/wi/wausau/profile/payday-loans/xpress-cash-0694-44153054
Found URL: https://www.bbb.org/us/wi/madison/profile/payday-loans/solomon-finance-inc-0694-44254104
Found URL: https://www.bbb.org/us/wi/marshfield/profile/payday-loans/check-advance-0694-44261229
Found URL: https://www.bbb.org/us/wi/appleton/profile/payday-loans/mister-money-usa-of-appleton-0694-25003191
Found URL: https://www.bbb.org/us/il/naperville/profile/payday-loans/bizcap-funding-0654-88650321
Found URL: https://www.bbb.org/us/il/oak-park/profile/loans/americas-financial-choice-inc-0654-88704524
Found URL: https://www.bbb.org/us/il/elgin/profile/loans/cmk-investments-inc-0654-88019672
Found URL: https://www.bbb.org/us/il/elgin/profile/loans/cmk-investments-inc-0654-88019672
Found URL: https://www.bbb.org/us/il/elgin/profile/loans/cmk-investments-inc-0654-88019672
Found URL: https://www.bbb.org/us/il

/var/folders/w8/dbs1z3013dnc63k_n9_0r0jc0000gn/T/ipykernel_26467/3044269838.py:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  business_started = soup.find('dt', text='Business Started Locally:').find_next_sibling('dd').text.strip() if soup.find('dt', text='Business Started Locally:') else 'N/A'
/var/folders/w8/dbs1z3013dnc63k_n9_0r0jc0000gn/T/ipykernel_26467/3044269838.py:46: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phone = soup.find('a', href=True, text=lambda x: x and x.startswith('(')).text.strip() if soup.find('a', href=True, text=lambda x: x and x.startswith('(')) else 'N/A'


Scraping data from: https://www.bbb.org/us/wi/wausau/profile/payday-loans/xpress-cash-0694-44153054
Scraping data from: https://www.bbb.org/us/wi/madison/profile/payday-loans/solomon-finance-inc-0694-44254104
Scraping data from: https://www.bbb.org/us/wi/marshfield/profile/payday-loans/check-advance-0694-44261229
Scraping data from: https://www.bbb.org/us/wi/appleton/profile/payday-loans/mister-money-usa-of-appleton-0694-25003191
Scraping data from: https://www.bbb.org/us/il/naperville/profile/payday-loans/bizcap-funding-0654-88650321
Scraping data from: https://www.bbb.org/us/il/oak-park/profile/loans/americas-financial-choice-inc-0654-88704524
Scraping data from: https://www.bbb.org/us/il/elgin/profile/loans/cmk-investments-inc-0654-88019672
Scraping data from: https://www.bbb.org/us/il/elgin/profile/loans/cmk-investments-inc-0654-88019672
Scraping data from: https://www.bbb.org/us/il/elgin/profile/loans/cmk-investments-inc-0654-88019672
Scraping data from: https://www.bbb.org/us/il/

...

Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/carlsbad/profile/loans/checkmate-1126-1022699
Found URL: https://www.bbb.org/us/ca/boulevard/profile/payday-loans/greenstream-lending-1126-172008103
Found URL: https://www.bbb.org/us/ca/irvine/profile/payday-loans/advance-payday-inc-1126-13084402
Found URL: https://www.bbb.org/us/ca/san-diego/profi

If prompted, please solve the CAPTCHA and press Enter to continue... 


Found URL: https://www.bbb.org/us/al/birmingham/profile/check-cashing-services/check-depot-0463-60001159
Found URL: https://www.bbb.org/us/al/birmingham/profile/check-cashing-services/check-depot-0463-60001159
Found URL: https://www.bbb.org/us/al/tuscaloosa/profile/payday-loans/cash-4-u-net-0463-90005947
Found URL: https://www.bbb.org/us/nh/claremont/profile/payday-loans/usa-pay-day-loans-0051-92023903
Found URL: https://www.bbb.org/us/al/livingston/profile/payday-loans/cash-2-go-0463-90104723
Found URL: https://www.bbb.org/us/al/selma/profile/title-loans/speedee-cash-0463-90145476
Found URL: https://www.bbb.org/us/al/foley/profile/payday-loans/emergi-cash-0463-90149144
Found URL: https://www.bbb.org/us/al/foley/profile/payday-loans/emergi-cash-0463-90149144
Found URL: https://www.bbb.org/us/fl/belleview/profile/payday-loans/cash-kwik-florida-inc-0733-90197157
Found URL: https://www.bbb.org/us/fl/winter-springs/profile/debt-repayment-plan/real-payday-loan-help-0733-90206048
Found URL: 

If prompted, please solve the CAPTCHA and press Enter to continue... 


Found URL: https://www.bbb.org/us/fl/fort-myers/profile/advance-fee-brokers/direct-loan-company-0653-90319824
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/loans/check-city-1086-76110
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/loans/check-city-1086-76110
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/loans/check-city-1086-76110
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/loans/check-city-1086-76110
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/loans/check-city-1086-76110
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/loans/check-city-1086-76110
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/loans/check-city-1086-76110
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/loans/check-city-1086-76110
Found URL: https://www.bbb.org/us/nv/las-vegas/profile/payday-loans/vip-cash-fast-1086-90011372
Found URL: https://www.bbb.org/us/nv/henderson/profile/payday-loans/ilenderast-1086-90011894
Found URL: https://www.bbb.org/us/nv/